## Information

This notebook was written and meant to run with the collab GPU for training - however for testing it can be ran locally.

In [ ]:
# Import torch libraries
import torch
import transformers
import torchaudio
import librosa
from torch import nn

# Importing other libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  # Library for progress bars

# Set device = GPU if available // Most important for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Set base dir
base_dir = "/content/drive/MyDrive/Arash Projects"

# Load dataset
